# Preprocessing Energy and Climate Data

* This notebook uses a dataset available at the provided link https://zindi.africa/competitions/ibm-skillsbuild-hydropower-climate-optimisation-challenge/data

* The dataset includes variables related to energy production, consumption, and climate. Each time series is recorded at 5-minute intervals.

* The objective of this notebook is to aggregate the energy and climate data to a daily resolution.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import polars as pl

In [2]:
path_data = 'hydropower-zindi-challenge/'

In [3]:
df_data = pl.read_csv(path_data+'Data.csv')
df_climate = pd.read_excel(path_data+'Kalam Climate Data.xlsx')

df_data.shape, df_climate.shape

In [ ]:
df_data.head()

date_time,v_red,current,power_factor,kwh,Source,v_blue,v_yellow,consumer_device_9,consumer_device_x
str,f64,f64,f64,f64,str,str,str,i64,i64
"""2024-07-22 18:20:00""",137.65,0.08,0.72,0.000661,"""consumer_device_10_data_user_1""",null,null,0,10
"""2024-07-22 18:25:00""",122.82,0.08,0.73,0.000598,"""consumer_device_10_data_user_1""",null,null,0,10
"""2024-07-22 18:30:00""",119.7,0.08,0.74,0.000591,"""consumer_device_10_data_user_1""",null,null,0,10
"""2024-07-22 18:35:00""",124.53,0.08,0.75,0.000623,"""consumer_device_10_data_user_1""",null,null,0,10
"""2024-07-22 18:40:00""",134.84,0.08,0.74,0.000665,"""consumer_device_10_data_user_1""",null,null,0,10


## Preprocess energy data

In [4]:
## cast numerical variables
df_data = df_data.with_columns([pl.col("v_blue").cast(pl.Float64),
                                pl.col("v_yellow").cast(pl.Float64)])

## Convert the 'date_time' column to a Polars Datetime type
df_data = df_data.with_columns(pl.col("date_time").str.strptime(pl.Datetime, "%Y-%m-%d %H:%M:%S").alias("date_time"))
## Extract derived date, hour, dayifweek
df_data = df_data.with_columns(
    pl.col("date_time").dt.date().alias("date"),
    pl.col("date_time").dt.hour().alias("hour"),
    pl.col("date_time").dt.weekday().alias("dayofweek")
)
## is weekend
df_data = df_data.with_columns(
    pl.col("dayofweek").is_in([5, 6]).cast(pl.Int8).alias("is_weekend")
)

## dividing user and devices
df_data = df_data.with_columns(
    pl.col("Source").alias('source_original'),
    pl.col("Source").str.split("_data_user_").list.to_struct(fields=["consumer_device", "data_user"])
).unnest("Source")

In [5]:
## aggregation of energy
df_daily_energy = df_data.group_by(['source_original', 'date']).agg(
    pl.col('kwh').sum()
)

## aggregationf of other energy related columns
df_data_agg = df_data.group_by(['source_original', 'date']).agg(
    [
        pl.col("v_red").mean().alias("v_red_mean"),
        pl.col("v_red").std().alias("v_red_std"),
        pl.col("v_red").median().alias("v_red_median"),

        pl.col("v_blue").mean().alias("v_blue_mean"),
        pl.col("v_blue").std().alias("v_blue_std"),
        pl.col("v_blue").median().alias("v_blue_median"),

        pl.col("v_yellow").mean().alias("v_yellow_mean"),
        pl.col("v_yellow").std().alias("v_yellow_std"),
        pl.col("v_yellow").median().alias("v_yellow_median"),

        pl.col("current").mean().alias("current_mean"),
        pl.col("current").std().alias("current_std"),
        pl.col("current").median().alias("current_median"),

        pl.col("power_factor").mean().alias("power_factor_mean"),
        pl.col("power_factor").std().alias("power_factor_std"),
        pl.col("power_factor").min().alias("power_factor_median"),
    ]
)

## merge aggegated enery and other related columns in one pandas df
merged_df = df_data_agg.join(df_daily_energy, on=['source_original', 'date'], how="inner").to_pandas()
merged_df.shape

## Preprocess climate data

In [6]:
## preprocess climate data
df_climate['date'] = df_climate['Date Time'].dt.date
df_climate.columns = ['date_time', 'temp', 'dew_temp', 'u_wind', 'v_wind', 'total_precip', 'snowfall', 'snow_cover', 'date']

df_climate_agg = df_climate.groupby(['date']).agg({
    'temp': ['mean', 'median', 'std'],
    'dew_temp': ['mean', 'median', 'std'],
    'u_wind': ['mean', 'median', 'std'],
    'v_wind': ['mean', 'median', 'std'],
    'total_precip': ['mean', 'median', 'std', 'sum'],
    'snowfall': ['mean', 'median', 'std', 'sum'],
    'snow_cover': ['mean', 'median', 'std'],
}).reset_index()
df_tmp = pd.DataFrame(np.vstack(df_climate_agg.columns.values))
df_tmp['columns_name'] = df_tmp[0] + '_' + df_tmp[1]
df_climate_agg.columns = list(df_tmp[0] + '_' + df_tmp[1].values)
df_climate_agg['date_'] = pd.to_datetime(df_climate_agg['date_'])
df_climate_agg.shape